
# Tune side model

In [7]:
import lightgbm as lgb
import optuna
import pandas as pd
from jesse import helpers, research
from sklearn.metrics import roc_auc_score

from custom_indicators.all_features import feature_bundle
from offline.labeling.labeling import TripleBarrierLabeler, expand_labels

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
warmup_3m, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
warmup_15m, trading_15m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "15m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

features_1m = feature_bundle(trading_1m, sequential=True)
features_1m = pd.DataFrame(
    {f"1m_{k}": v for k, v in features_1m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_1m[:, 0]]),
)
features_1m = features_1m.resample("3T").agg({k: "last" for k in features_1m.columns})
features_3m = feature_bundle(trading_3m, sequential=True)
features_3m = pd.DataFrame(
    {f"3m_{k}": v for k, v in features_3m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_3m[:, 0]]),
)
features_15m = feature_bundle(trading_15m, sequential=True)
features_15m = pd.DataFrame(
    {f"15m_{k}": v for k, v in features_15m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_15m[:, 0]]),
)
features_15m = (
    features_15m.resample("3T").agg({k: "last" for k in features_15m.columns}).ffill()
)

df_features = pd.concat([features_1m, features_3m, features_15m], axis=1)
print(df_features.shape)


def get_side_label(candles, num_minutes, target_ret, pt_sl):
    labeler = TripleBarrierLabeler(candles, num_minutes=num_minutes, verbose=False)
    side_labels = labeler.side_labels(pt=pt_sl, sl=pt_sl, target_ret=target_ret)
    candle_df = expand_labels(side_labels, candles, fill=0)
    return candle_df

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(818400, 1263)


In [8]:
df_features.to_parquet("data/features.parquet")

In [ ]:
def objective(trial):
    num_minutes = trial.suggest_int("num_minutes", 10, 120)
    target_ret = trial.suggest_float("target_ret", 0.0005, 0.01)
    pt_sl = trial.suggest_float("pt_sl", 0.1, 1)
    label_df = get_side_label(trading_3m, num_minutes, target_ret, pt_sl)

    train_test_split_point = int(len(label_df) * 0.75)
    train_features = df_features.iloc[:train_test_split_point]
    test_features = df_features.iloc[train_test_split_point:]
    train_labels = label_df.iloc[:train_test_split_point]
    test_labels = label_df.iloc[train_test_split_point:]

    train_valid_mask = train_labels["bin"].astype(int) != 0
    test_valid_mask = test_labels["bin"].astype(int) != 0

    train_features = train_features[train_valid_mask]
    train_labels = (train_labels[train_valid_mask]["bin"].astype(int) == 1).astype(int)
    test_features = test_features[test_valid_mask]
    test_labels = (test_labels[test_valid_mask]["bin"].astype(int) == 1).astype(int)
    if test_labels.nunique() <= 1:
        return 0

    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 100,
        "boosting": "gbdt",
        "num_leaves": 80,
        "max_depth": 50,
    }
    dtrain = lgb.Dataset(train_features, train_labels)
    dtest = lgb.Dataset(test_features, test_labels)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=800,
    )
    pred_proba = model.predict(test_features)
    auc = roc_auc_score(test_labels, pred_proba, average="weighted")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, show_progress_bar=True, n_jobs=1)

In [ ]:
from jesse import research, helpers
from offline.labeling.labeling import TripleBarrierLabeler, expand_labels

warmup_3m, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

num_minutes = 10
pt_sl = 0.9656765406603607
target_ret = 0.005978786042921383
labeler = TripleBarrierLabeler(trading_3m, num_minutes=10, verbose=True)
side_labels = labeler.side_labels(pt=pt_sl, sl=pt_sl, target_ret=target_ret)
print(side_labels["bin"].value_counts().sort_index().to_numpy())
candle_df = expand_labels(side_labels, trading_3m, fill=0)

In [6]:
candle_df.to_parquet("data/label_side.parquet")

# Tune meta model

In [ ]:
import lightgbm as lgb
import optuna
import pandas as pd
from jesse import helpers, research
from sklearn.metrics import roc_auc_score

from custom_indicators.all_features import feature_bundle
from custom_indicators.selection import SIDE_1M, SIDE_3M, SIDE_15M
from offline.labeling.labeling import TripleBarrierLabeler, expand_labels

warmup_1m, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
warmup_3m, trading_3m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)
warmup_15m, trading_15m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "15m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

features_1m = feature_bundle(trading_1m, sequential=True)
features_1m = pd.DataFrame(
    {f"1m_{k}": v for k, v in features_1m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_1m[:, 0]]),
)
features_1m = features_1m.resample("3T").agg({k: "last" for k in features_1m.columns})
features_3m = feature_bundle(trading_3m, sequential=True)
features_3m = pd.DataFrame(
    {f"3m_{k}": v for k, v in features_3m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_3m[:, 0]]),
)
features_15m = feature_bundle(trading_15m, sequential=True)
features_15m = pd.DataFrame(
    {f"15m_{k}": v for k, v in features_15m.items()},
    index=pd.DatetimeIndex([helpers.timestamp_to_time(i) for i in trading_15m[:, 0]]),
)
features_15m = (
    features_15m.resample("3T").agg({k: "last" for k in features_15m.columns}).ffill()
)

df_features = pd.concat([features_1m, features_3m, features_15m], axis=1)
print(df_features.shape)

side_model = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_features_col = SIDE_1M + SIDE_3M + SIDE_15M
side_model_pred = side_model.predict(df_features[side_features_col])
df_features["side_model_res"] = side_model_pred


def get_meta_label(candles, num_minutes, target_ret, pt, sl):
    labeler = TripleBarrierLabeler(candles, num_minutes=num_minutes, verbose=False)
    side_labels = labeler.side_labels(pt=pt, sl=sl, target_ret=target_ret)
    side_full_df = expand_labels(side_labels, candles)
    side_full_df["pred"] = [1 if i > 0.5 else -1 for i in side_model_pred]
    meta_labels = labeler.meta_labels(side_full_df, pt=pt, sl=sl, target_ret=target_ret)
    candle_df = expand_labels(meta_labels, candles, fill=0)
    return candle_df


def objective(trial):
    num_minutes = trial.suggest_int("num_minutes", 10, 120)
    target_ret = trial.suggest_float("target_ret", 0.0005, 0.01)
    pt = trial.suggest_float("pt", 0.1, 1.2)
    sl = trial.suggest_float("sl", 0.1, 1.2)
    label_df = get_meta_label(trading_3m, num_minutes, target_ret, pt, sl)

    train_test_split_point = int(len(label_df) * 0.8)
    train_features = df_features.iloc[:train_test_split_point]
    test_features = df_features.iloc[train_test_split_point:]
    train_labels = label_df.iloc[:train_test_split_point]
    test_labels = label_df.iloc[train_test_split_point:]

    train_valid_mask = train_labels["ret"].notna().tolist()
    test_valid_mask = test_labels["ret"].notna().tolist()

    train_features = train_features[train_valid_mask]
    train_labels = train_labels[train_valid_mask]["bin"].astype(int)
    test_features = test_features[test_valid_mask]
    test_labels = test_labels[test_valid_mask]["bin"].astype(int)
    if test_labels.nunique() <= 1:
        return 0

    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "early_stopping_round": 100,
        "boosting": "gbdt",
        "num_leaves": trial.suggest_int("num_leaves", 31, 200),
        "max_depth": trial.suggest_int("max_depth", 30, 100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 1),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 1),
    }
    dtrain = lgb.Dataset(train_features, train_labels)
    dtest = lgb.Dataset(test_features, test_labels)
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 300, 800),
    )
    pred_proba = model.predict(test_features)
    auc = roc_auc_score(test_labels, pred_proba, average="weighted")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=500, show_progress_bar=True, n_jobs=1)

(891360, 1263)


/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_83944/433169081.py:133: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2025-02-19 23:21:18,754] A new study created in memory with name: no-name-ffba4aa2-c20c-49c2-acbd-294ca7506757
Best trial: 0. Best value: 0.659896:   0%|          | 1/500 [02:52<23:53:33, 172.37s/it]

[I 2025-02-19 23:24:11,125] Trial 0 finished with value: 0.6598955325939089 and parameters: {'num_minutes': 78, 'target_ret': 0.006563809745103193, 'pt': 0.8412562428293201, 'sl': 0.4065178704446605, 'num_leaves': 114, 'max_depth': 78, 'min_gain_to_split': 0.5458092139221663, 'min_data_in_leaf': 38, 'lambda_l1': 0.21896224192056243, 'lambda_l2': 0.7090538280989217, 'num_boost_round': 604}. Best is trial 0 with value: 0.6598955325939089.


Best trial: 1. Best value: 0.675298:   0%|          | 2/500 [05:36<23:10:36, 167.54s/it]

[I 2025-02-19 23:26:55,289] Trial 1 finished with value: 0.6752980630471858 and parameters: {'num_minutes': 46, 'target_ret': 0.007477193510141419, 'pt': 0.29984117045998443, 'sl': 0.5037397907615571, 'num_leaves': 171, 'max_depth': 87, 'min_gain_to_split': 0.41273612709249047, 'min_data_in_leaf': 67, 'lambda_l1': 0.7315359626470875, 'lambda_l2': 0.40994685016876936, 'num_boost_round': 712}. Best is trial 1 with value: 0.6752980630471858.


...

[I 2025-02-20 09:13:30,975] Trial 331 finished with value: 0.869731063159189 and parameters: {'num_minutes': 14, 'target_ret': 0.009659712854172822, 'pt': 0.9513658477182422, 'sl': 0.7709850811657137, 'num_leaves': 58, 'max_depth': 79, 'min_gain_to_split': 0.19486417899960767, 'min_data_in_leaf': 98, 'lambda_l1': 0.866815178359078, 'lambda_l2': 0.6600511753066319, 'num_boost_round': 601}. Best is trial 252 with value: 0.9112837356943453.


Best trial: 252. Best value: 0.911284:  67%|██████▋   | 333/500 [9:53:46<4:18:47, 92.98s/it]

[I 2025-02-20 09:15:05,005] Trial 332 finished with value: 0.8750105995081828 and parameters: {'num_minutes': 12, 'target_ret': 0.009500871455250299, 'pt': 0.9907280275126014, 'sl': 0.8541998962833923, 'num_leaves': 53, 'max_depth': 88, 'min_gain_to_split': 0.6148262359955532, 'min_data_in_leaf': 114, 'lambda_l1': 0.8835099785797408, 'lambda_l2': 0.7323781908883344, 'num_boost_round': 609}. Best is trial 252 with value: 0.9112837356943453.


Best trial: 252. Best value: 0.911284:  67%|██████▋   | 334/500 [9:55:23<4:20:34, 94.18s/it]

[I 2025-02-20 09:16:41,990] Trial 333 finished with value: 0.7239541478120813 and parameters: {'num_minutes': 49, 'target_ret': 0.009996320500264532, 'pt': 0.9192050225945907, 'sl': 0.9197277658367315, 'num_leaves': 62, 'max_depth': 85, 'min_gain_to_split': 0.6465965458760876, 'min_data_in_leaf': 104, 'lambda_l1': 0.8439143910428628, 'lambda_l2': 0.6824762203571568, 'num_boost_round': 623}. Best is trial 252 with value: 0.9112837356943453.
